# What this notebook does

The overall goal is produce a table that can be used to create a feature-set ready for model training. To do that, we fist pull commit information about a github project that has the url listed in <code>project + '_url.txt'</code>, then format that info as a table where each row is a separate commit and columns hold info about the commit. Here is the target, where each row represents a single commit:

<img src="https://www.dropbox.com/s/c2wrow809najo9s/Screenshot%202019-07-15%2011.11.08.png?raw=1">

After this we will build a second table that compacts infomation into days. This will result in a table that looks like this:

<img src="https://www.dropbox.com/s/53ddnk7201mmecz/Screenshot%202019-07-15%2011.09.43.png?raw=1">

## Assumptions of this notebook

1. The code and notebooks we will be using, including this one, are in the following folder: <code>/ideas-uo/machine_learning/predicting_project_activity</code>.

2. We expect to be able to execute this code: <code>project_url = project + '_url.txt'</code> and then <code>project_info = open(os.path.join('.',project_url),'r').readlines()</code>. Two lines are expected in the file: the github url of the repo and the name of the repo. Other lines will be ignored.   

3. Two tables will be written to <code>'/ideas-uo/machine_learning/predicting_project_activity'</code>: (1) <code>project +  '_commits_table.csv'</code> and (2) <code>project +  '_days_table.csv'</code>.

4. This notebook should be started in folder <code>/ideas-uo/machine_learning/predicting_project_activity</code>.


### Aside

This notebook was originally broken into 2: one that produced commit table and one that produced day table. Because the 2 tables are strongly linked, they were combined into the single notebook.

## Parameters for this notebook

In [1]:
project = 'spack'  #or latte, etc.

# Part 1

Our goal is to load in and parse project data and produce a pandas dataframe that is commit-based, i.e., every row represents a separate commit.

## Do github pull

Assume that notebook started in context of ideas-uo repository.


In [2]:
import os
starting_dir = os.getcwd() #~/ideas-uo/machine_learning/predicting_project_activity
starting_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/machine_learning/predicting_project_activity'

In [3]:
#~/ideas-uo/machine_learning/predicting_project_activity
%pwd

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/machine_learning/predicting_project_activity'

In [4]:
#move up to uo-ideas folder - must be better way
%cd ../..

/Users/fickas/Dropbox/boyana/my_work/ideas-uo


In [5]:
repository_dir = os.getcwd() #~/ideas-uo/
repository_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo'

In [6]:
!git fetch --all

Fetching origin


In [7]:
#Refresh local repository
!git pull

Already up-to-date.


In [8]:
code_dir = repository_dir + '/code'
code_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code'

In [9]:
import sys
sys.path.append(code_dir)

In [10]:
print( '\n'.join(sys.path))


/Users/fickas/anaconda2/envs/py36/lib/python36.zip
/Users/fickas/anaconda2/envs/py36/lib/python3.6
/Users/fickas/anaconda2/envs/py36/lib/python3.6/lib-dynload
/Users/fickas/anaconda2/envs/py36/lib/python3.6/site-packages
/Users/fickas/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/extensions
/Users/fickas/.ipython
/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code


## Our library

We wrote the hpcl library to act as a github puller. See /ideas-uo/code/hpcl/ for source.

In [11]:
import os, sys, subprocess
from hpcl import Command
from hpcl import GitCommand
#import mysql.connector    #works from command line but not here


In [12]:
#move back to code dir /ideas-uo/code

os.chdir(code_dir)
os.getcwd()

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code'

## Note uses "project name"_url.txt to get projects to load

Typically only want to focus on a single project.

Pulling the project info can take some time.

In [13]:
project_url = project + '_url.txt'# Where to get the project from github

In [14]:
#Produces nested Python lists and dictionaries

def checkoutSubrepos(repos,tdir):
    #currdir = os.getcwd()
    for repopath in repos.keys():
        if repopath == tdir: continue
        retcode, out, err = Command.Command('git checkout %s%s' % repos[repopath]).run(dryrun=dry_run)
    return


if __name__ == "__main__":
    
    print('Going to load repos...')

    #Load the repo to clone from the following file
    project_info = open(os.path.join('.',project_url),'r').readlines()

    #Move to the tmp directory that will hold the repo
    currdir = os.getcwd()
    tmpdir = os.path.join(currdir,'tmp')  #sff: fails if does not already exist
    #os.chdir(tmpdir)

    commander = GitCommand.GitCommand(tmpdir)

    #Download repo
    url = project_info[0]
    name = project_info[1]
    if name != project: print(('warning: names are different', name, project))
    print('Cloning: ' + name)
    commander.cloneRepo(url)

    print('Finished cloning repo.')

    #Setup DB connection, this will probably change soon.
    #mydb = mysql.connector.connect(host="localhost", port="3307", user="pythondb", passwd="********", database="gitstats")
    #print(mydb)
    #mycursor = mydb.cursor()
    #sql = "INSERT INTO stats (reponame, stats) VALUES(%s, %s)"

    #Now run commands on the repo

    if not os.path.isdir(name): print(('missing repo', name))

    print('Checking: ' + name)
    prefix,versions = commander.getRepoVersions(name)
    print(prefix)
    print(versions)

    #Gets all the commit data in a dictionary format {authorname:{'total_commits':0, 'commits':{'date':'', 'message':'', 'diffs':{'filename':'', 'diff':''}}}}
    active_developers = commander.getRepoCommitData(name)

    print(('number of developers', len(active_developers)))

    """ stats = getStats('.',repo)
    ts = repo + ', ' + out.strip().split('-')[0] 
    buf = ts
    for i in range(0,len(category_names)): 
        buf += ', %d' % stats[category_names[i]]
        linecounts[i].append(stats[category_names[i]]*0.001)
    outfile.write(buf+'\n')
    #years.append(int(ts))
    print(stats)

    val = (repo, buf)"""
    #mycursor.execute(sql,val)
    #mydb.commit()






Going to load repos...
Cloning: spack
Finished cloning repo.
Checking: spack

[b'cc76c0f5f9f8021cfb7423a226bd431c00d791ce', b'30d3b32085ab31c91ac45f2f14c5de07774823f7', b'0a0291678e283bf154081df67b0a1f5c909d1d19', b'f5a49280c3b9063c6deb29307cd6356bf75cedd5', b'34d23c617c89861e0d4ee1aad6a0acf1892502df', b'c3b003e6984f15807fe8675d90eab19679566363', b'8540d5390e70388625ee006562b450efa924113b']
git checkout b'cc76c0f5f9f8021cfb7423a226bd431c00d791ce'
b''
git checkout b'30d3b32085ab31c91ac45f2f14c5de07774823f7'
b''
git checkout b'0a0291678e283bf154081df67b0a1f5c909d1d19'
b''
git checkout b'f5a49280c3b9063c6deb29307cd6356bf75cedd5'
b''
git checkout b'34d23c617c89861e0d4ee1aad6a0acf1892502df'
b''
git checkout b'c3b003e6984f15807fe8675d90eab19679566363'
b''
git checkout b'8540d5390e70388625ee006562b450efa924113b'
b''
('number of developers', 569)


## Let's see what we have

In [15]:
len(active_developers)

569

In [16]:
dev_tups = list(active_developers.items())  #get it in form easier to see

## Nested structure is complicated

Code below dives into the structure for illustration. It can be skipped on production run.

In [17]:
a_dev = dev_tups[0]  #record for one developer with commits buried in that record
a_dev

(b'Massimiliano Culpo <massimiliano.culpo@gmail.com>',
 {'commits': [{'date': b'Mon Jul 15 19:30:01 2019 +0200',
    'diffs': [{'diff': ['+import spack.error'],
      'filename': 'a/lib/spack/spack/cmd/uninstall.py b/lib/spack/spack/cmd/uninstall.py'}],
    'id': b'5acbe449e5840a7592e93d3ba35ff10e45ebc8a0',
    'message': '    spack uninstall can uninstall specs with multiple roots (#11977)\n    \n    Fixes #3690\n    Fixes #5637\n    \n    Uninstalling dependents of a spec was relying on a traversal of the\n    parents done by inspecting spec._dependents. This is in turn a\n    DependencyMap that maps a package name to a single DependencySpec object\n    (an edge in the DAG) and cannot thus model the case where a spec has\n    multiple configurations of the same parent package installed (for\n    example if different versions of the same Python library depend on\n    the same Python installation).\n    \n    This commit works around this issue by constructing the list of specs to\n   

In [18]:
a_dev[0]  #developer name

b'Massimiliano Culpo <massimiliano.culpo@gmail.com>'

In [19]:
a_dev[1].keys()  #dictionary with 2 keys and we only care about commits

dict_keys(['total_commits', 'commits'])

In [20]:
a_dev[1]['total_commits']

973

In [21]:
list_of_commits = a_dev[1]['commits']  #expect a separate row for each commit in this list
len(list_of_commits )

973

In [22]:
list_of_commits [0]  #dictionary of date, diffs, id, message - this corresponds to one row in the table

{'date': b'Mon Jul 15 19:30:01 2019 +0200',
 'diffs': [{'diff': ['+import spack.error'],
   'filename': 'a/lib/spack/spack/cmd/uninstall.py b/lib/spack/spack/cmd/uninstall.py'}],
 'id': b'5acbe449e5840a7592e93d3ba35ff10e45ebc8a0',
 'message': '    spack uninstall can uninstall specs with multiple roots (#11977)\n    \n    Fixes #3690\n    Fixes #5637\n    \n    Uninstalling dependents of a spec was relying on a traversal of the\n    parents done by inspecting spec._dependents. This is in turn a\n    DependencyMap that maps a package name to a single DependencySpec object\n    (an edge in the DAG) and cannot thus model the case where a spec has\n    multiple configurations of the same parent package installed (for\n    example if different versions of the same Python library depend on\n    the same Python installation).\n    \n    This commit works around this issue by constructing the list of specs to\n    be uninstalled in an alternative way, and adds tests to verify the\n    behavior

In [23]:
list_of_commits[0].keys()

dict_keys(['id', 'date', 'message', 'diffs'])

In [24]:
actual_diffs = list_of_commits [0]['diffs'] #a list of dictionaries with keys diff, filename
actual_diffs

[{'diff': ['+import spack.error'],
  'filename': 'a/lib/spack/spack/cmd/uninstall.py b/lib/spack/spack/cmd/uninstall.py'}]

In [25]:
list_of_diff_strings = actual_diffs[0]['diff']  #a list of strings - have to parse to pull out info
list_of_diff_strings

['+import spack.error']

### Aside

The comlicated nesting of information takes a bit to understand. Drawing out the structure is on our to-do list.

## Goal reminder

Goal is to reformat the data. Currently the key is user and the value is a nested structure that includes all the commits for that person. We want to invert this. We'd like a list of commits, where each commit has info including developer name.

Note Python is a mess when it comes to time zones. What is supposed to work, <code>%z</code>, does not. See discussion here: https://stackoverflow.com/questions/3305413/python-strptime-and-timezones.

In particular, these fail when include UTC: <code>'%a %b %m %X %y %z'</code> and <code>'%c %z'</code>.

We are resorting to a utility package.

In [26]:
import datetime
from dateutil import parser

In [27]:
x = parser.parse("Tue Aug 2 14:58:31 2016 +0200")  #+0200 hours => 7200 seconds
x  #just an example

datetime.datetime(2016, 8, 2, 14, 58, 31, tzinfo=tzoffset(None, 7200))

In [28]:
x.utcoffset().seconds

7200

## Helper function

Given a list of diff strings (see list_of_diff_strings above), do a count of lines of code changed for each string. At moment this is trivial given we assume each string represents a single change (+ or -). So in theory we could just take the length of change_list as locc. However, we want to verify that each string starts with a + or - and print a warning if not.
<p>
Eventually we want to get back to looking at replacing a line with another one. This will be counted as 2 changes, a + and a -, when it should only be counted as 1. On our to-do list is to further process change_list to catch these type of replace actions.

In [29]:
import re  #use in future

In [30]:
#This is a trivial method until we start thinking about replace actions
def count_changes(change_list):
    change_count = 0
    #Go through each change represented as a string, e.g., '+ share/spack/qa/.*'
    for change in change_list:
        #Big assumption: single + or - per change and it is the first char
        if change[0] in '+-':
            change_count += 1
        else:
            print(('missing +- at index 0', change))
    #TODO: worry about a replace being counted as 2 instead of 1
    return change_count
        

## Create a list of dicts

Each dict will end up as a row in a commit-based table.

<code>entry</code> below is the 2-tuple version of a single developer record. It has 2 items, the developer name and the complicated stuff.

In [31]:
all_commits = []

for entry in dev_tups:
    name = entry[0]                               #'Gilles Fourestey <gilles.fourestey@epfl.ch>'
    commit_list = entry[1]['commits']      #a list of dictionaries with date, diffs, message as keys
    seen = []                               #need this because getting duplicates
    for a_commit in commit_list:
        if a_commit in seen: continue
        if not a_commit['diffs']: continue  #skip commit with empty diff
        seen.append(a_commit)
        new_dict = {}
        date = parser.parse(a_commit['date'])         #"Tue Aug 2 14:58:31 2016 +0200"
        new_dict['year'] = date.year
        new_dict['doy'] = min(365, int(date.strftime('%j'))) #punting on leap year
        new_dict['month'] = date.month
        new_dict['day_of_month'] = date.day
        new_dict['day_name'] = date.strftime('%A')  #e.g., Tuesday
        new_dict['utc_offset'] = date.utcoffset().seconds  #e.g., 7200

        new_dict['name'] = name
        new_dict['message'] = a_commit['message']
        
        diffs = a_commit['diffs']  #list of dictionaries diff, filename
        
        filenames = []
        changes_total = 0
        for real_diff in diffs:   #a dictionary diff filename
            filenames.append(real_diff['filename'])
            the_diffs = real_diff['diff']  #list of strings
            if the_diffs:
                change_count = count_changes(the_diffs)
                changes_total += change_count
            else:
                 print(('warning: empty diff list for', name, date.month, date.day, a_commit['message'], a_commit['id']))
        new_dict['filenames'] = filenames
        new_dict['locc'] = changes_total
        all_commits.append(new_dict)
        
    

('warning: empty diff list for', b'Massimiliano Culpo <massimiliano.culpo@gmail.com>', 5, 3, '    Added a function that concretizes specs together (#11158)\n    \n    * Added a function that concretizes specs together\n    \n    * Specs concretized together are copied instead of being referenced\n    \n    This makes the specs different objects and removes any reference to the\n    fake root package that is needed currently for concretization.\n    \n    * Factored creating a repository for concretization into its own function\n    \n    * Added a test on overlapping dependencies\n', b'5ffb27071405c77f825444577fb3066ed32b200c')
('warning: empty diff list for', b'Massimiliano Culpo <massimiliano.culpo@gmail.com>', 12, 19, '    Added labels in github issue templates (#10128)\n', b'e0575000098f6af00d7316948e205c462c910520')
('warning: empty diff list for', b'Massimiliano Culpo <massimiliano.culpo@gmail.com>', 7, 24, '    tests: Add python 3.7 to Travis CI (#8778)\n    \n    * Test Spack o

('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 8, 10, '    Spack packages now PEP8 compliant.\n', b'240f1fd2239552f37d0f8a8841496414b163ef10')
('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 7, 18, '    flake8 fixes.\n    \n    - package.py\n    - spec.py\n    - test/architecture.py\n', b'cddaba8add41bc93659070f1ffd18ffc9a0dabb0')
('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 7, 15, '    Fix broken `spack info` command after build dep merge.\n    \n    - Added a method to get dependencies of particular types from Package\n    - Fixed info command.\n', b'73f10c9363e584708a39cf02758fc837dc02b8d3')
('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 6, 27, '    Add new sys_type method that prints out the *full* arch descriptor.\n', b'99c3ed56fae3b4a70a03a60aa645a282136a025d')
('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 5, 10, '    Update code style checking.\n    \n    - Exempt

('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 8, 16, "    Consolidate most module code into spack.modules and spack.cmd.module\n    \n    - One file with all the module classes (spack/modules.py)\n      - Has an EnvModule superclass that does most of the work and consolidates common code\n      - Subclasses have specializations for different module systems (TclModule, Dotkit)\n    \n    - One command (spack module) for all the types of modules to use\n      - the one command is used by the scripts, only need to maintain in one place\n      - has some subcommands for different module types, but they're handled mostly generically.\n    \n    - Consolidate zsh support into a single setup-env.sh script.\n", b'221cf6acb932442f5964243eb3413a1d921920c8')
('warning: empty diff list for', b'Todd Gamblin <tgamblin@llnl.gov>', 7, 31, "    Add Kevin's experimental TAU version\n", b'90cd0c7efa9cc6b1e58530bede25deeaaaf9f52e')
('warning: empty diff list for', b'Todd Gamblin <t

('warning: empty diff list for', b'Adam J. Stewart <ajstewart426@gmail.com>', 1, 17, '    Support scionlib version detection (#2818)\n', b'061076aea92a1d29058f17505b820e27bda7d846')
('warning: empty diff list for', b'Adam J. Stewart <ajstewart426@gmail.com>', 12, 25, '    Modify github archive regex to support luaposix (#2677)\n', b'c3cd948537dc357d4a5e227e9c2f8e85a7a7342d')
('warning: empty diff list for', b'Adam J. Stewart <ajstewart426@gmail.com>', 11, 28, "    Fix building latest version of flex (#2401)\n    \n    * Fix building latest version of flex\n    \n    * Don't need when clause\n    \n    * Remove perl deps, shebang too long\n", b'664939eeba8d086106e6154a897214d2da7fe09d')
('warning: empty diff list for', b'Adam J. Stewart <ajstewart426@gmail.com>', 10, 5, '    Add package for CONVERGE CFD (#1905)\n', b'6ee020cea4040546c4faffc122ec9166868a09ab')
('warning: empty diff list for', b'Adam J. Stewart <ajstewart426@gmail.com>', 10, 5, "    Add missing dependencies for Flake8 (#1

('warning: empty diff list for', b'Mark W. Krentel <krentel@rice.edu>', 5, 24, "    mbedtls: add pic variant (#11558)\n    \n    Add variant 'pic' for including in shared libraries.\n    \n    Add build dependency on perl.  Compiling mbedtls runs several perl\n    scripts.\n    \n    Add versions 2.7.10 and 2.16.1, the heads of the two main stable\n    branches.\n", b'7c32a4c39951dd1d138fa8a33cdf695c404e53e2')
('warning: empty diff list for', b'Mark W. Krentel <krentel@rice.edu>', 1, 9, "    elfutils: option to install elf.h header file (#10185)\n    \n    * elfutils: option to install elf.h header file\n    \n    Add variant 'elfh' (default False) to install the elf.h header file to\n    the include directory.  This provides access to a newer version of\n    elf.h and allows an elf application to build with the same elf.h that\n    libelf.so was built with.\n    \n    Add version 0.175.\n    \n    * Drop the variant 'elfh' and always install elf.h instead.\n", b'1b991f9385e01b9530a35e

...

('warning: empty diff list for', b'Veselin Dobrev <v-dobrev@users.noreply.github.com>', 3, 6, '    mumps: add the libs property (#7264)\n', b'65a165502263cab2f22083c0b31df1e9a9631ce9')
('warning: empty diff list for', b'Christoph Junghans <christoph.junghans@gmail.com>', 4, 2, '    macsio: fix install (#11088)\n', b'4ff3ce494ac41ce7ad7b0946cd005010f8f1dc30')
('warning: empty diff list for', b'Christoph Junghans <christoph.junghans@gmail.com>', 7, 21, '    votca: add more deps (#8771)\n    \n    * votca: add more deps\n    \n    * Update package.py\n', b'0e4909cad8dbc243f1dc3f3c2926f282e8d9f15f')
('warning: empty diff list for', b'Christoph Junghans <christoph.junghans@gmail.com>', 7, 9, '    Add ccache support (#3761)\n    \n    If the user sets "ccache: true" in spack\'s config.yaml, Spack will use an available\n    ccache executable when compiling c/c++ code. This feature is disabled by default\n    (i.e. "ccache: false") and the documentation is updated with how to enable\n    ccach

('warning: empty diff list for', b'Jim Galarowicz <jeg@krellinst.org>', 8, 3, '    Fix flake8 errors for: Update libmonitor to use its new github location, it does not exist where the package file now points to.\n', b'2e1dbd0697ef2a07154ed83a0d31d144c1bb8b5e')
('warning: empty diff list for', b'Jim Galarowicz <jeg@krellinst.org>', 8, 3, '    Fix flake8 errors for Update the version of xerces-c, where the developers remove the old version, so there is no fallback.\n', b'e69423a154b54c5e2605442d97ab9454028e60d2')
('warning: empty diff list for', b'Jim Galarowicz <jeg@krellinst.org>', 4, 13, '    Loosen the boost requirement from a specific version that was specified in my initial commit.\n', b'f9aafeee1e6a33a76dc1904e97f117194915f4d0')
('warning: empty diff list for', b'Jim Galarowicz <jeg@krellinst.org>', 7, 21, '    Add qt3 version support and a variant that controls patching the qt3 version for compile issues and answering yes to the license questions during build.\n', b'15ec3cb3cd713

('warning: empty diff list for', b'serbanmaerean <serban@us.ibm.com>', 1, 4, "    Set SMPI library path in dependent packages (#6548)\n    \n    * Added magma package\n    \n    * Incorporated Serban's change\n    \n    * Remove deleted files\n    \n    * argparse.py\n    \n    * Sync forked Spack repo in github\n    \n    * Add spectrum-mpi lib path to LD_LIBRAY_PATH in spack_env.\n    As a result of supporting alternate installation directories, spectrum-mpi does\n    not use rpath anymore, so we need to set LD_LIBRARY_PATH in spack_env to allow\n    MPI programs to run during dependent package configuration.\n    \n    * Fix flake8 error (blank line at EOF)\n", b'8f6935d53f7f272286e154d52fe972e2249958e9')
('warning: empty diff list for', b'Matthew Scott Krafczyk <krafczyk.matthew@gmail.com>', 11, 7, '    Fix for nghttp2 installation (#5904)\n    \n    nghttp2 depends on python, and requires the install directory to exist,\n    and contain the python module directory which it will ev

('warning: empty diff list for', b'Erik Schnetter <schnetter@gmail.com>', 1, 8, '    Ensure than OpenBLAS provides a file liblapack.a\n    \n    OpenBLAS implements the virtual package "lapack", and its consumers apparently expect a "liblapack.a".\n', b'd9f1a55c902ca4544e99d946a81df33331f989dc')
('warning: empty diff list for', b'Michael Kuron <mkuron@users.noreply.github.com>', 1, 6, "    Siesta: only apply patch to <4.1 and fix Intel compiler (#6466)\n    \n    * Siesta: add 3.2 and don't apply patch to 4.1\n    \n    * siesta: fix bad optimization with Intel compiler\n    \n    * siesta: Fix download URL\n    \n    * siesta: build utilities too\n    \n    * siesta: fix binary name collision\n    \n    * siesta: Cleanup\n", b'38fabce2aabbf5a131b4ca056187bc5afe092983')
('warning: empty diff list for', b'Audrey Thoma <las_thoma15@iastate.edu>', 7, 7, '    fastx-toolkit: solving gcc conflict (#4696)\n', b'56b959501aac18b18abf713c122132d065e849fa')
('warning: empty diff list for', b'Mass

('warning: empty diff list for', b'citibeth <rpf2116@columbia.edu>', 4, 1, '    Added documentation for installing Environment Modules with Spack.\n', b'459aab628d73c7f6f1d4eecbf64b34ca334e0ada')
('warning: empty diff list for', b'citibeth <rpf2116@columbia.edu>', 3, 5, '    Fixed URLs\n    Removed vestigal mpi variant in netcdf-fortran\n', b'ae87948a232c17982a7987e5423ac77acdc59194')
('warning: empty diff list for', b'Elizabeth F <rpf2116@columbia.edu>', 4, 27, '    spack setup: Fix broken module convenience settings.\n', b'4d466fe879820e18f18e96809f6825d393792de3')
('warning: empty diff list for', b'Elizabeth F <rpf2116@columbia.edu>', 3, 25, '    Added documentation for StagedPackage, etc.\n', b'9889b1a5febc2aedec6401ae701b55961cf0d559')
('warning: empty diff list for', b'Elizabeth F <rpf2116@columbia.edu>', 3, 4, '    1. Removed fake MPI dependency from netcdf-cxx4 and netcdf-fortran.\n    \n    2. Removed Fortran bootstrap variant from netcdf.  Users who need NetCDF Fortran interf

In [32]:
len(all_commits)

10646

In [33]:
all_commits[:5]

[{'day_name': 'Monday',
  'day_of_month': 15,
  'doy': 196,
  'filenames': ['a/lib/spack/spack/cmd/uninstall.py b/lib/spack/spack/cmd/uninstall.py'],
  'locc': 1,
  'message': '    spack uninstall can uninstall specs with multiple roots (#11977)\n    \n    Fixes #3690\n    Fixes #5637\n    \n    Uninstalling dependents of a spec was relying on a traversal of the\n    parents done by inspecting spec._dependents. This is in turn a\n    DependencyMap that maps a package name to a single DependencySpec object\n    (an edge in the DAG) and cannot thus model the case where a spec has\n    multiple configurations of the same parent package installed (for\n    example if different versions of the same Python library depend on\n    the same Python installation).\n    \n    This commit works around this issue by constructing the list of specs to\n    be uninstalled in an alternative way, and adds tests to verify the\n    behavior. The core issue with DependencyMap is not resolved here.\n',
  'mo

Looks ok to me. Now converting to pandas dataframe is easy peasy.

In [34]:
import pandas as pd

In [48]:
raw_table = pd.DataFrame(all_commits)  #gotta love pandas

In [49]:
commits_table = raw_table[['month', 'day_of_month', 'day_name', 'year',
                           'locc', 'message', 'filenames', 'name', 'doy', 'utc_offset']]

In [50]:
commits_table.head()

,month,day_of_month,day_name,year,locc,message,filenames,name,doy,utc_offset
0,7,15,Monday,2019,1,spack uninstall can uninstall specs with m...,[a/lib/spack/spack/cmd/uninstall.py b/lib/spac...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,196,7200
1,6,13,Thursday,2019,3,"Make ""spack compiler find"" check PATH by d...",[a/lib/spack/spack/cmd/compiler.py b/lib/spack...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,164,7200
2,6,7,Friday,2019,2,Compiler search uses a pool of workers (#1...,[a/lib/spack/llnl/util/filesystem.py b/lib/spa...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,158,7200
3,5,28,Tuesday,2019,10,Cap the maximum number of build jobs (#113...,[a/etc/spack/defaults/config.yaml b/etc/spack/...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,148,7200
4,5,24,Friday,2019,6,build env: simplify handling of parallel j...,[a/lib/spack/spack/build_environment.py b/lib/...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,144,7200


In [51]:
#df_sorted = df_commits.sort_values(by='date')  #fails because of UTC
commits_table = commits_table.sort_values(['year', 'doy'])  #primary and secondary
commits_table = commits_table.reset_index(drop=True)

In [52]:
commits_table.head(50)

,month,day_of_month,day_name,year,locc,message,filenames,name,doy,utc_offset
0,2,13,Wednesday,2013,5,Initial version of spack with one package:...,"[a/.gitignore b/.gitignore, a/bin/spack b/bin/...",b'Todd Gamblin <tgamblin@llnl.gov>',44,57600
1,2,18,Monday,2013,2,Require python2.7\n,[a/bin/spack b/bin/spack],b'Todd Gamblin <tgamblin@llnl.gov>',49,57600
2,2,18,Monday,2013,2,"Dependencies now work. Added libelf, libd...",[a/bin/spack b/bin/spack],b'Todd Gamblin <tgamblin@llnl.gov>',49,57600
3,2,19,Tuesday,2013,3,Fixed passing of dependence prefixes to cc...,[a/lib/spack/env/cc b/lib/spack/env/cc],b'Todd Gamblin <tgamblin@llnl.gov>',50,57600
4,2,19,Tuesday,2013,6,"Fixes, remove parallel build for libdwarf ...",[a/lib/spack/env/cc b/lib/spack/env/cc],b'Todd Gamblin <tgamblin@llnl.gov>',50,57600
5,2,19,Tuesday,2013,1,"rpaths for dependencies. elf, dwarf, cmak...",[a/lib/spack/env/cc b/lib/spack/env/cc],b'Todd Gamblin <tgamblin@llnl.gov>',50,57600
6,2,20,Wednesday,2013,2,Fixed bug in parallel make option.\n,[a/lib/spack/spack/Package.py b/lib/spack/spac...,b'Todd Gamblin <tgamblin@llnl.gov>',51,57600
7,2,20,Wednesday,2013,1,Added libunwind and fixed link issues in c...,[a/lib/spack/env/cc b/lib/spack/env/cc],b'Todd Gamblin <tgamblin@llnl.gov>',51,57600
8,2,21,Thursday,2013,4,Better handling of stage.\n - better sy...,[a/lib/spack/spack/Package.py b/lib/spack/spac...,b'Todd Gamblin <tgamblin@llnl.gov>',52,57600
9,2,21,Thursday,2013,2,Parallel bootstrap for cmake.\n,[a/lib/spack/spack/Package.py b/lib/spack/spac...,b'Todd Gamblin <tgamblin@llnl.gov>',52,57600


In [39]:
for x in dev_tups:
    if x[0] == b'Todd Gamblin <tgamblin@llnl.gov>':
        found = x
        break

In [40]:
z = found[1]['commits']
z

[{'date': b'Mon Jul 15 07:32:51 2019 -0700',
  'diffs': [{'diff': ['-    homepage = "https://computation.llnl.gov/projects/co-design/amg2013"',
     '+    homepage = "https://computing.llnl.gov/projects/co-design/amg2013"'],
    'filename': 'a/var/spack/repos/builtin/packages/amg/package.py b/var/spack/repos/builtin/packages/amg/package.py'}],
  'id': b'a3caf52cac9f19638220b837e4870210d698da0d',
  'message': '    packages: computation.llnl.gov is now computing.llnl.gov (#12013)\n'},
 {'date': b'Fri Jul 12 08:33:23 2019 -0700',
  'diffs': [{'diff': ['-    # This logic is derived from the cea-hpc/modules profile.sh example at',
     '-    # https://github.com/cea-hpc/modules/blob/master/init/profile.sh.in',
     '-    #',
     '-    # The objective is to correctly detect the shell type even when setup-env',
     '-    # is sourced within a script itself rather than a login terminal.',
     '-        echo ${BASH##*/}',
     '+        echo bash',
     '-        echo $ZSH_NAME',
     '+    

In [41]:
oct11 = []
for i,d in enumerate(z):
    if b'Oct 11' in d['date'] and 'pydoc' in d['message']:
        oct11.append(d['diffs'])
oct11

[[{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}],
 [{'diff': [],
   'filename': 'a/lib/spack/spack/cmd/doc.py b/lib/spack/spack/cmd/doc.py'}]]

In [42]:
z[435]

{'date': b'Tue Oct 11 23:13:40 2016 -0700',
 'diffs': [{'diff': ["-        spack.curl.add_default_arg('-k')",
    '+        spack.insecure = True'],
   'filename': 'a/bin/spack b/bin/spack'}],
 'id': b'488e1bab2ca384863517375f47be33dca4f170f8',
 'message': '    Make `insecure` option work with curl AND git. (#1786)\n'}

In [53]:
commits_table.tail()

,month,day_of_month,day_name,year,locc,message,filenames,name,doy,utc_offset
10641,7,14,Sunday,2019,5,bzip2: Add 1.0.8 (#12017)\n \n Updat...,[a/var/spack/repos/builtin/packages/bzip2/pack...,b'Michael Kuhn <michael.kuhn@informatik.uni-ha...,195,7200
10642,7,15,Monday,2019,1,spack uninstall can uninstall specs with m...,[a/lib/spack/spack/cmd/uninstall.py b/lib/spac...,b'Massimiliano Culpo <massimiliano.culpo@gmail...,196,7200
10643,7,15,Monday,2019,2,packages: computation.llnl.gov is now comp...,[a/var/spack/repos/builtin/packages/amg/packag...,b'Todd Gamblin <tgamblin@llnl.gov>',196,61200
10644,7,16,Tuesday,2019,0,binutils: added '-Wno-narrowing' to CXXFLA...,[a/var/spack/repos/builtin/packages/binutils/p...,b'Hironori-Yamaji <52182908+Hironori-Yamaji@us...,197,32400
10645,7,16,Tuesday,2019,3,py-basemap: install without egg (#11961)\n...,[a/var/spack/repos/builtin/packages/py-basemap...,b'Milton Woods <miltonjwoods@gmail.com>',197,36000


In [54]:
len(commits_table)

10646

In [45]:
foo_fie_fum()  #break here from Run All and decide if want to write out

NameError: name 'foo_fie_fum' is not defined

Write it out to file

In [55]:
save_dir = repository_dir + '/machine_learning/predicting_project_activity/'

In [56]:
commits_table.to_csv(save_dir + project +  '_commits_table.csv', index=False)

# Part 2

Now produce a day-based table using the df_sorted table as a starting point.

In [57]:
import datetime
from dateutil import parser

## Here is starting date

starting_year = commits_table.loc[0,'year']
starting_month = commits_table.loc[0,'month']
starting_day = commits_table.loc[0,'day_of_month']
starting_obj = datetime.date(starting_year, starting_month, starting_day)
starting_obj

datetime.date(2013, 2, 13)

In [58]:
## Here is ending date

ending_year = commits_table.iloc[-1]['year']
ending_month = commits_table.iloc[-1]['month']
ending_day = commits_table.iloc[-1]['day_of_month']
ending_obj = datetime.date(ending_year, ending_month, ending_day)
ending_obj

datetime.date(2019, 7, 16)

In [59]:
#We should end up with a list of this length, i.e., a list item for each day.

td = ending_obj - starting_obj
td.days

2344

## Approach

Goal: for everyday between starting and ending dates, create a row for that day.

Actual method: loop through rows of commit table. Keep values needed to (a) count rows with same date, (b) count days skipped leading to a sequence of 0 entries, and (c) determine when switch years so can reset values.

Note that things like messages and filenames will be combined into a list that goes with a day, i.e., can have mulitple filenames and multiple messages associated with the activity in one day.

In [68]:
#Fill out this new table. These columns will be the fodder for features in a feature set.

days_table = pd.DataFrame(columns= ['month', 'day_of_month', 'day_of_week', 'year', 'total_commits',
                           'total_loccs', 'total_messages', 'total_filenames', 'total_names',
                                    'doy'])


In [69]:
current_day = int(commits_table.loc[0,'doy'])  #day of year: 1-365 (or 366 on leap years)
current_year = commits_table.loc[0,'year']
dnint = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}

names = []
messages = []
filenames = []
total_commits = 0
total_loccs = 0


for i in range(len(commits_table)):

    #pull out date pieces
    year = int(commits_table.loc[i,'year'])
    day_of_year = int(commits_table.loc[i,'doy'])
    
    #check if change years, e.g., change from 2013 to 2014
    if year!=current_year:
        current_year = year
        diff = day_of_year + (365 - current_day)  #account for skipped days at end of old year
    else:
        diff = day_of_year - current_day
    
    #diff now holds number of days incremented
    
    #No diff so same day - increment all tracked values for the day
    if diff==0:
        total_loccs += commits_table.loc[i,'locc']
        total_commits += 1
        messages.append(commits_table.loc[i,'message'])
        names.append(commits_table.loc[i,'name'])
        filenames.append(commits_table.loc[i,'filenames'])
        continue
    
    #Now things get interesting. We need to move back in time to beginning edge of gap. If gap is size diff,
    #then move back diff days. That will give us the date before the gap begins.
    
    #First build date object - easier to do arithmetic on. This is date on ending edge of gap.
    month = int(commits_table.loc[i,'month'])
    day_of_month = int(commits_table.loc[i,'day_of_month'])
    end_gap_date = datetime.datetime(year, month, day_of_month)   #current row we are looking at
    
    begin_gap_date = end_gap_date - datetime.timedelta(days=diff) #looking back in time
    
    #record row values for begin gap date
    prior_day_name = dnint[begin_gap_date.strftime('%A')]  #convert to int 1-7
    prior_month = begin_gap_date.month
    prior_day_of_month = begin_gap_date.day
    prior_year = begin_gap_date.year
    prior_doy = min(365, int(begin_gap_date.strftime('%j')))

    #build row and append
    new_row = {'day_of_week': prior_day_name, 'month': prior_month, 'day_of_month': prior_day_of_month,
               'doy': prior_doy, 'year': prior_year,
               'total_messages': messages, 'total_names': names, 'total_filenames': filenames,
               'total_loccs': total_loccs, 'total_commits': total_commits}
    days_table = days_table.append(new_row, ignore_index=True)
    
    #Whew. Took care of recording data for the beginning data of gap.
    
    #diff = 1 so tomorrow is here :) Just reset things since no dates skipped
    if diff == 1:
        total_loccs = commits_table.loc[i,'locc']
        total_commits = 1
        messages = [commits_table.loc[i,'message']]
        names = [commits_table.loc[i,'name']]
        filenames = [commits_table.loc[i,'filenames']]
        current_day = day_of_year
        continue
    
    #we have a gap! need to fill in with 0 feature values for each day in gap
    if diff > 1:
        date_obj = begin_gap_date
        total_loccs = 0
        total_commits = 0
        messages = []
        names = []
        filenames = []
        for j in range(diff-1):
            date_obj += datetime.timedelta(days=1)  #handles month change overs
            day_name = dnint[date_obj.strftime('%A')]
            new_row = {'day_of_week': day_name, 'month': date_obj.month, 'day_of_month': date_obj.day,
               'doy': min(365, int(date_obj.strftime('%j'))), #punting on leap year
                'year': date_obj.year, 'total_names': names, 'total_filenames': filenames,
               'total_messages': messages, 'total_loccs': total_loccs, 'total_commits': total_commits}
            days_table = days_table.append(new_row, ignore_index=True)
        #record the new commit we just saw for data at end of gap
        total_loccs = commits_table.loc[i,'locc']
        total_commits = 1
        messages = [commits_table.loc[i,'message']]
        names = [commits_table.loc[i,'name']]
        filenames = [commits_table.loc[i,'filenames']]
        current_day = day_of_year  #now on new date
        continue
    
    print((i, day_of_year, year, diff))
    raise Exception  #should never get here

#check if have any values accumulated.
if total_commits>0:
    final_commit_row = commits_table.iloc[-1]
    end_date = datetime.datetime(final_commit_row['year'], final_commit_row['month'], final_commit_row['day_of_month'])
    new_row = {'day_of_week': dnint[end_date.strftime('%A')], 'month': end_date.month, 'day_of_month': end_date.day,
               'doy': final_commit_row['doy'], 'year': end_date.year,
               'total_messages': messages, 'total_names': names, 'total_filenames': filenames,
               'total_loccs': total_loccs, 'total_commits': total_commits}
    days_table = days_table.append(new_row, ignore_index=True) 

In [70]:
days_table.head(50)

,month,day_of_month,day_of_week,year,total_commits,total_loccs,total_messages,total_filenames,total_names,doy
0,2,13,3,2013,1,5,[ Initial version of spack with one package...,"[[a/.gitignore b/.gitignore, a/bin/spack b/bin...",[b'Todd Gamblin <tgamblin@llnl.gov>'],44
1,2,14,4,2013,0,0,[],[],[],45
2,2,15,5,2013,0,0,[],[],[],46
3,2,16,6,2013,0,0,[],[],[],47
4,2,17,7,2013,0,0,[],[],[],48
5,2,18,1,2013,2,4,"[ Require python2.7\n, Dependencies now...","[[a/bin/spack b/bin/spack], [a/bin/spack b/bin...","[b'Todd Gamblin <tgamblin@llnl.gov>', b'Todd G...",49
6,2,19,2,2013,3,10,[ Fixed passing of dependence prefixes to c...,"[[a/lib/spack/env/cc b/lib/spack/env/cc], [a/l...","[b'Todd Gamblin <tgamblin@llnl.gov>', b'Todd G...",50
7,2,20,3,2013,2,3,"[ Fixed bug in parallel make option.\n, ...",[[a/lib/spack/spack/Package.py b/lib/spack/spa...,"[b'Todd Gamblin <tgamblin@llnl.gov>', b'Todd G...",51
8,2,21,4,2013,6,16,[ Better handling of stage.\n - better s...,[[a/lib/spack/spack/Package.py b/lib/spack/spa...,"[b'Todd Gamblin <tgamblin@llnl.gov>', b'Todd G...",52
9,2,22,5,2013,1,9,[ Better spack -h: added cmd descriptions.\...,[[a/bin/spack b/bin/spack]],[b'Todd Gamblin <tgamblin@llnl.gov>'],53


In [66]:
len(days_table)

2344

In [ ]:
fee_fie_foe()  #stop here before writing. Will stop Run All here.

In [67]:
days_table.to_csv(save_dir + project +  '_days_table.csv', index=False)